# **Script and outputs for training a CNN for model selection, perform cross-validation and predict the most likely model from empirical data in the species *Euphorbia segueriana*.**
From the manuscript Kirschner & Perez et al. (2022) "Congruent evolutionary responses of European steppe biota to late Quaternary climate change: insights from convolutional neural network-based demographic modeling".

All required files are available at the GitHub directory, except for the simulations, that can be downloaded here: https://drive.google.com/file/d/1GdTSSu_RwWAljqA8dpj-FW1juiojhJqb/view?usp=sharing. We will do the download with the next command.

In [ ]:
#You also need to download the empirical data.
!wget https://raw.githubusercontent.com/ColonnaLab/EMBO_popgen/main/popgen2024/Manolo_Perez/Par1_Dem_Models_Esegueriana.txt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1GdTSSu_RwWAljqA8dpj-FW1juiojhJqb' -O simulations.zip

In [ ]:
# Connect the notebook with your Google Drive.

from google.colab import drive
drive.mount('/content/drive')

##**Section 1: Building the CNN.**

In [ ]:
# Import all required modules.

import sys, os
import numpy as np
import time
import random
from random import shuffle, choice
import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import load_model
#from keras.utils import np_utils
import sklearn.metrics as metrics
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In the next cell we define the CNN architecture. Try to recognize each part of the architecture and associate them to the concepts presented during the lecture and what are their functions. Take notes if there is anything new or if you are not sure of what is the function of any of these parts.

In [ ]:
# Define parameters for the CNN run.
batch_size = 200
### how much interations to train the network
epochs = 100

###n of models
num_classes = 3


# Define the CNN architecture.
def create_cnn(xtest):
	inputShape = (xtest.shape[1], xtest.shape[2])
	## image size. images need to have EXACTLY the same size
	inputs = Input(shape=inputShape)
	x = inputs
	## 1D convolution - less computational intensive and is also invariant to the samples order;
	x = Conv1D(250, kernel_size=2, activation='relu',input_shape=(xtest.shape[1], xtest.shape[2]))(x)
	### Enables the network to learn more complex features / shapes.
	x = AveragePooling1D(pool_size=2)(x)
	x = BatchNormalization()(x)
	x = Conv1D(125, kernel_size=2, activation='relu')(x)
	### Enables the network to learn more complex features / shapes.
	x = AveragePooling1D(pool_size=2)(x)
	x = BatchNormalization()(x)
	x = Conv1D(125, kernel_size=2, activation='relu')(x)
	### Enables the network to learn more complex features / shapes.
	x = AveragePooling1D(pool_size=2)(x)
	x = BatchNormalization()(x)
	### Linearising the image as in the initial step. From this point on the network behaves as a Multi-Layer Perceptron.
	x = Flatten()(x)
	x = Dense(125, activation='relu')(x)
	x = Dropout(0.5)(x)
	x = Dense(125, activation='relu')(x)
	x = Dropout(0.5)(x)
	x = Dense(num_classes, activation="softmax")(x)

	# Construct the CNN.
	model = Model(inputs, x)
	# Return the CNN.
	return model

## **Section 2: Train the network with 1,000 simulations from each model**

Now we are going to load the simulated data and train the network. First, unzip the folder containing the simulations and load them as 3-dimensional NumPy arrays (simulation, SNPs, samples). Associate each simulation with the appropriate label and shuffle their order.

In [ ]:
#Unzip simularions
!unzip "/content/drive/MyDrive/simulations.zip"

#Load simulations from each model as a separate NumPy array
u1 = np.load("/content/simulations/trainingSims/simModel1.npy",mmap_mode='r')
u2 = np.load("/content/simulations/trainingSims/simModel2.npy",mmap_mode='r')
u3 = np.load("/content/simulations/trainingSims/simModel3.npy",mmap_mode='r')

# Combine all arrays.
x=np.concatenate((u1,u2,u3),axis=0)

#For the real data we don't know which alleles are ancestral or derived.
#This function to transforms major alleles in -1 and minor in 1
for arr,array in enumerate(x):
  for idx,row in enumerate(array):
    if np.count_nonzero(row) > len(row)/2:
      x[arr][idx][x[arr][idx] == 1] = -1
      x[arr][idx][x[arr][idx] == 0] = 1
    else:
      x[arr][idx][x[arr][idx] == 0] = -1

# Label each simulated array.
y=[0 for i in range(len(u1))]
y.extend([1 for i in range(len(u2))])
y.extend([2 for i in range(len(u3))])
y = np.array(y)

#delete temporary files to free memory.
del (u1,u2,u3)

# Shuffle the arrays for training, keeping the labels in the same order.
shf = list(range(len(x)))
shuffle(shf)
#Shuffle the labels (models)
y = y[shf]
#Shuffle the simulations using the same order of the labels.
x = x[shf]


In [ ]:
# Print label and simulations length, these should be the same.
print (len(x), len(y))

# Check array sizes as well just to be sure.
print (x.shape)

print (y.shape)

Now let's visualize the simulations as images. Check if you can extract some information from the images. For example, is it possible to separate individuals from the two different populations (remember that each sample is in a different column, and that the population samples are separated).

In [ ]:
#Print the label and export an image from the training data, to visualize its appearance
print(y[0])
import matplotlib.pyplot as plt
plt.imshow(x[0],cmap='gray', vmin=-1, vmax=1)

In [ ]:
#You can also visualize a subset of the SNPs to better see the polymorphism.
import matplotlib.pyplot as plt
plt.imshow(x[0][0:200],cmap='gray', vmin=-1, vmax=1)

In [ ]:
#you can also visualize the array values (-1 for major and 1 to minor).
x[0]

Now we need to make the simulations look more like the real data. For that, we need to convert ancestral/derived alleles into major/minor. We also need to add missing data to the matrices.

In [ ]:
# We will add missing data to the simulations as 0s.
# This is necessary beacuse the real data contains missing genotypes and
# we need to train the network to be able to recognize it.

#Add missing data (coded as 0s) to the simulated matrices
# as a percentage according to the empirical data (15% in E. segueriana).
missD_perc = 15
missD = int(x.shape[1]*x.shape[2]*(missD_perc/100))
for i in range(x.shape[0]):
    indices_2d = np.random.choice(x.shape[1], size=missD, replace=True)
    indices_3d = np.random.choice(x.shape[2], size=missD, replace=True)
    x[i, indices_2d, indices_3d] = 0
del(missD)

In [ ]:
#Export an image from the training data, to visualize its appearance.
#Note how missing data (in grey) make it much more difficult to detect patterns.
import matplotlib.pyplot as plt
plt.imshow(x[0][0:200],cmap='gray', vmin=-1, vmax=1)

In [ ]:
#And visualize the array values.
x[0]

Now we need to separate the training simulations into the training and validation sets. Do you remember what are these sets used to? We don't need to define the test now, but do you remember what will it be used to?

We will also compile the architecture, define the optimzer and set an Early Stopping approach.

In [ ]:
# Separate train (75%) and validate (25%) sets.
xtrain, xtest = x[int(len(y)*.25):], x[:int(len(y)*.25)]
ytrain, ytest = y[int(len(y)*.25):], y[:int(len(y)*.25)]
del(x)
ytest = np.eye(num_classes)[ytest]
ytrain = np.eye(num_classes)[ytrain]

# Create the CNN network.
model = create_cnn(xtest)

# Compile the CNN.
model.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer='Adam',
	              metrics=['accuracy'])

# We will use early stopping and save the model with the best val_accuracy.
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=50, verbose=0, mode='max', restore_best_weights=True)
### stop training when validation error increases (wait 50 epochs to see if there is any improvement).

# Check the architecture.
model.summary()


In [ ]:
#We can also have a graphical visualization of the layers
!pip install git+https://github.com/paulgavrikov/visualkeras --upgrade
import visualkeras
from PIL import ImageFont
visualkeras.layered_view(model, legend=True)

No it's time to train the network. This will take some time. While it's training we can discuss the previous steps of the script.

In [ ]:
#Run the CNN
history = model.fit(xtrain, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(xtest, ytest),callbacks=[earlyStopping])

# Save the model.
model.save(filepath='Trained_Esegueriana._MissingData.acc.mod')

After training we need to evaluate if the network is learning and not overfitting. The next commands will plot the accuracy in the training and validation sets.

In [ ]:
# Plot training and validation accuracies
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper left')
plt.show()

What do you think about this curves? Do you think that training was properly done? Can you think of ways to improve it?

## **Section 3: Perform cross validation predictions with another 100 simulations per model. Those were not seen by the network during training**

Now we are going to test the trained model using the test set. These are new simulations that were not used by the network during the training. The idea here is to use simulaiton as if they are our real data (we call this pseudoobserved data in ABC) and see what the network predicts for each of these simulations. After that we compare the predicted values with the real (simulated) ones.

In [ ]:
# Load Numpy test set arrays containing the test set simulations.
u1 = np.load("/content/simulations/testSims/simModel1.npy",mmap_mode='r') #Expansion in both
u2 = np.load("/content/simulations/testSims/simModel2.npy",mmap_mode='r') #Expansion in Zonal only
u3 = np.load("/content/simulations/testSims/simModel3.npy",mmap_mode='r') #No expansion

# Combine all test set arrays.
xtest=np.concatenate((u1,u2,u3),axis=0)

#transform major alleles in -1 and minor 1
for arr,array in enumerate(xtest):
  for idx,row in enumerate(array):
    if np.count_nonzero(row) > len(row)/2:
      xtest[arr][idx][xtest[arr][idx] == 1] = -1
      xtest[arr][idx][xtest[arr][idx] == 0] = 1
    else:
      xtest[arr][idx][xtest[arr][idx] == 0] = -1

# Label each simulated array.
ytest=[0 for i in range(len(u1))]
ytest.extend([1 for i in range(len(u2))])
ytest.extend([2 for i in range(len(u3))])
ytest = np.array(ytest)

#delete temporary files to free memory.
del (u1,u2,u3)

# Print label and simulations length, these should be the same.
print (len(xtest), len(ytest))

#Add missing data (coded as 0s) to the simulated matrices
# as a percentage according to the empirical data (15% in E. segueriana).
missD_perc = 15
missD = int(xtest.shape[1]*xtest.shape[2]*(missD_perc/100))
for i in range(xtest.shape[0]):
    indices_2d = np.random.choice(xtest.shape[1], size=missD, replace=True)
    indices_3d = np.random.choice(xtest.shape[2], size=missD, replace=True)
    xtest[i, indices_2d, indices_3d] = 0
del(missD)

# Predict with the trained model and export a confusion matrix.
pred = model.predict(xtest)
pred_cat = [i.argmax() for i in pred]
cm=confusion_matrix(ytest, pred_cat)
scenarios = ['Expansion_Z+ExZ', 'Expansion_Z', 'Stable']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=scenarios)

disp.plot(cmap=plt.cm.Blues)
plt.show()

Please take a moment to analyze the confusion matrix. Can you detect which models are more difficult to predict? For those, what is the other model that is creating this confusion?

## **Section 4: Predict the most likely model for the empirical data, using the trained CNN.**

In [ ]:
# Load empirical data.
infile=np.loadtxt("/content/Par1_Dem_Models_Esegueriana.txt")
inp=np.array(infile)

inp.shape

In [ ]:
# Create 100 subsets containing 1,000 random SNPs from the full empirical data.
num_samples=100
res = []
for i in range(0,num_samples):
	idx = np.random.choice(inp.shape[0], 1000, replace=False)
	n = inp[idx,:]
	res.append(np.array(n))

Emp = np.array(res)

# Predict model probabilities for each of the 100 subsets.
Emp_pred = model.predict(Emp)

# Print for each subset
print(Emp_pred)

In [ ]:
# Print the average of all subsets
np.mean(Emp_pred, axis=0)


Which scenario was selected? Is this in agreement with the one obtained in the paper for *E. segueriana?* (model 2 - Zonal expansion only). Check with your colleagues whether they recovered the same scenario as you. If not, can you explain why this is the case?

## **Section 5: We can also perform parameter estimation.**

Now we can use the same architecture (with a slight difference in the last layer) to perform parameter estimation. To make things simple, we will focus only in the simulations under the selected scenario (model 2). We will also need to change the labels of each simulation to the parameter values used.

In [ ]:
# Define a function to read the parameters file.
def readDemogParams(demogParamPath):
    params = []
    first = True
    with open(demogParamPath) as demogParamFile:
        for line in demogParamFile:
            params.append([float(x) for x in line.strip().split()])
    return params

#define the architecture.
def create_cnn(xtest):
	inputShape = (xtest.shape[1], xtest.shape[2])
	## image size. images need to have EXACTLY the same size
	inputs = Input(shape=inputShape)
	x = inputs
	## 1D convolution - less computational intensive and also treats snps as independent;
	x = Conv1D(250, kernel_size=2, activation='relu',input_shape=(xtest.shape[1], xtest.shape[2]))(x)
	### Enables the network to learn more complex features / shapes.
	x = AveragePooling1D(pool_size=2)(x)
	x = BatchNormalization()(x)
	x = Conv1D(125, kernel_size=2, activation='relu')(x)
	### Enables the network to learn more complex features / shapes.
	x = AveragePooling1D(pool_size=2)(x)
	x = BatchNormalization()(x)
	x = Conv1D(125, kernel_size=2, activation='relu')(x)
	### Enables the network to learn more complex features / shapes.
	x = AveragePooling1D(pool_size=2)(x)
	x = BatchNormalization()(x)
	### Linearising the image as in the initial step. From this point on the network behaves as a Multi-Layer Perceptron.
	x = Flatten()(x)
	x = Dense(125, activation='relu',kernel_regularizer=l2(1e-3), bias_regularizer=l2(1e-3))(x)
	x = Dropout(0.5)(x)
	x = Dense(125, activation='relu',kernel_regularizer=l2(1e-3), bias_regularizer=l2(1e-3))(x)
	x = Dropout(0.5)(x)
	# The last layer is a dense according to the number of parameters.
	x = Dense(numParams)(x)# Define the CNN architecture.

	# Construct the CNN.
	model = Model(inputs, x)
	# Return the CNN.
	return model

In [ ]:
from sklearn.preprocessing import StandardScaler

# Load parameters.
demogParams = readDemogParams('/content/simulations/trainingSims/parameters.txt')
y = np.array(demogParams)
numParams=y.shape[1]

#delete temporary files to free memory.
del (demogParams)

#We need to standard scale the parameters, as they can be on very different scales.
scaler= StandardScaler()
y = scaler.fit_transform(y)

#Load simulations for the best model.
u3 = np.load("/content/simulations/trainingSims/simModel2.npy",mmap_mode='r')
x = np.array(u3)

#transform major alleles in -1 and minor 1
for arr,array in enumerate(x):
  for idx,row in enumerate(array):
    if np.count_nonzero(row) > len(row)/2:
      x[arr][idx][x[arr][idx] == 1] = -1
      x[arr][idx][x[arr][idx] == 0] = 1
    else:
      x[arr][idx][x[arr][idx] == 0] = -1

# Print label and simulations length, these should be the same.
print (len(x), len(y))

# Shuffle the arrays for training, keeping the labels in the same order.
shf = list(range(len(x)))
shuffle(shf)
y = y[shf]
x = x[shf]

#Add missing data (coded as 0s) to the simulated matrices
# as a percentage according to the empirical data (15% in E. segueriana).
missD_perc = 15
missD = int(x.shape[1]*x.shape[2]*(missD_perc/100))
for i in range(x.shape[0]):
    indices_2d = np.random.choice(x.shape[1], size=missD, replace=True)
    indices_3d = np.random.choice(x.shape[2], size=missD, replace=True)
    x[i, indices_2d, indices_3d] = 0
del(missD)

# Separate train (75%) and validate (25%) sets.
xtrain, xtest = x[int(len(y)*.25):], x[:int(len(y)*.25)]
ytrain, ytest = y[int(len(y)*.25):], y[:int(len(y)*.25)]
del(x)

# Create the CNN network.
cnn = create_cnn(xtest)

# Compile the CNN.
cnn.compile(loss='mean_squared_error', #the loss is now defined as the mean squared error, as we have continuous labels
	              optimizer='Adam')

# Check the architecture.
cnn.summary()

# Run the CNN with early stopping. Save the model with the best val_accuracy.
earlyStopping = EarlyStopping(monitor='val_loss', patience=50, verbose=0, mode='max', restore_best_weights=True)

history = cnn.fit(xtrain, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(xtest, ytest),callbacks=[earlyStopping])

In [ ]:
# Plot training and validation accuracies
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

##**Now let's use the test set.**


In [ ]:
# Load the simulations.
x_test = np.load("/content/simulations/testSims/simModel2.npy",mmap_mode='r')
x_test = np.array(x_test)

#transform major alleles in -1 and minor 1
for arr,array in enumerate(xtest):
  for idx,row in enumerate(array):
    if np.count_nonzero(row) > len(row)/2:
      xtest[arr][idx][xtest[arr][idx] == 1] = -1
      xtest[arr][idx][xtest[arr][idx] == 0] = 1
    else:
      xtest[arr][idx][xtest[arr][idx] == 0] = -1


#Add missing data (coded as 0s) to the simulated matrices
# as a percentage according to the empirical data (15% in E. segueriana).
missD_perc = 15
missD = int(xtest.shape[1]*xtest.shape[2]*(missD_perc/100))
for i in range(xtest.shape[0]):
    indices_2d = np.random.choice(xtest.shape[1], size=missD, replace=True)
    indices_3d = np.random.choice(xtest.shape[2], size=missD, replace=True)
    xtest[i, indices_2d, indices_3d] = 0
del(missD)


# Predict parameters for each simulation.
pred = cnn.predict(x_test)
#return predictions to their correct scale.
pred = scaler.inverse_transform(pred)

In [ ]:

# Create 100 subsets containing 1,000 random SNPs from the full empirical data.
num_samples=100
res = []
for i in range(0,num_samples):
	idx = np.random.choice(inp.shape[0], 1000, replace=False)
	n = inp[idx,:]
	res.append(np.array(n))

# Predict parameters.
Emp_pred = np.array(res)
Emp_pred = cnn.predict(Emp_pred)
Emp_pred = scaler.inverse_transform(Emp_pred)

#Print parameter names
print('Theta, T1, T2, T3, Ne, NeZ, NeExZLGM, NeZLGM, NeExZPl, NeZPl, m12_Pres, m21_Pres, m12_LGM, m21_LGM, m12_Pl, m21_Pl')
#Print parameter values (averaged over subsets)
print(np.mean(Emp_pred, axis=0))

Now you have just obtained an estimate of parameter values for the empirical data. Notice that these are point estimates (average over 100 subsets). Is this different from the type of results you obtained with ABC? Can you think of potential approaches to estimate uncertainty (feel free to try something below)?